<div style="text-align: center; font-size: 36px; font-weight: bold;">
    Project: Get, Clean, Update, & Export:
</div><br>  
<div style="font-size: 14px;"><span style="font-size: 18px;">Purpose:</span><br>
The goal of this project is to take a daily, multiple and complex step process down to just a couple steps. The script will automate cleaning,<br>
gather files from multiple locations, update the data, and redistribute the final Excel product back out to the respected locations. A key process<br>
is to ensure that any inputs from the user into the code created columns would copy over to the new dataframe. As a last step, it will format each of<br>
the Excel files so they are ready to view with no adjustments needed from the users.
</div>

## Project Flowchart:
<div style="font-size: 14px;">This outlines the workflow of the code.<br>The main steps are shown on the Left side.<br>The processes taking place is everything to the 
Right.</div>

![Flowchart](images/project_flowchart.png)

## Imported Libraries

In [ ]:
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment
import sys

## Welcome statement:  
<div style="font-size: 14px;">Confirmation forces a small pause before<br>the code starts to run.</div>

In [ ]:
while True:
        
    ask_to_run = input("Do you want to run this report? [Y]es / [N]o\n >>> ")
    if ask_to_run.lower().startswith('y'):
        break
    elif ask_to_run.lower().startswith('n'):
        exit()
    else:
        print('Response not recognized')
        print('Please enter [Y]es or [N]o\n')

## Get the new report  
<div style="font-size: 14px;">The new raw data is downloaded from its source and then placed in a file where the script can reach it.<br>
In this case, the variable: <i>new_report.</i> It will also set up a variable for a list that will be used later and variables to<br>
deal with the time formating when we clean.

In [ ]:
new_report = r"\\path\to\the_new_file\the_data_file.xlsx"

try:
    df = pd.read_excel(new_report, header=6)
except Exception as e:
    input(f"Failed to load: {e}\nPress Enter: Script will Terminate...")
    sys.exit(1)


concat_odf = [] # used later in the code
current_year = datetime.now().year
today = pd.to_datetime(datetime.now().date())

## Clean the new data
<div style="font-size: 14px;">This will keep relevant columns, filter relevant data, correct a column to properly show elapsed<br>
time that was broken in original data, and add 2 columns not part of the original dataframe.</div>

In [ ]:
keep_columns = ['keep_1', 'keep_2', 'keep_3', 'keep_4',
                'keep_5', 'keep_6', 'keep_7', 'keep_8', 'keep_9']
filter_status = ['orange']
filter_showroom = ['up', 'down']

df = df[keep_columns]
df = df[df['keep_9'].str.contains('|'.join(filter_status), case=False)]
df = df[~df['keep_2'].str.contains('|'.join(filter_showroom), case=False)]
df['new_col_1'] = ''
df['keep_3'] = pd.to_datetime(f'{current_year}/' + df['keep_3'], format='%Y/%m/%d')
df['keep_4'] = (today - df['keep_3']).dt.days
df['keep_3'] = df['keep_3'].dt.date
df['keep_3'] = pd.to_datetime(df['keep_3'])
df['keep_3'] = df['keep_3'].dt.strftime('%m/%d/%Y')
df = df.sort_values(by='keep_4', ascending=False)
df['new_col_2'] = None

## Start the update process
<div style="font-size: 14px;">This section will be where target file paths live. It will also go out and collect all the existing files and merge<br>
them into one df. This is to prepare the existing data to be updated with the new data.</div>

In [ ]:
odf1 = None
odf2 = None
odf3 = None
odf4 = None
odf5 = None
odf6 = None
odf7 = None
odf8 = None
odf9 = None

odf_lists = [odf1, odf2, odf3, odf4, odf5, odf6, odf7, odf8, odf9]
odf_filepaths = [
    r"\\local_1\the_folder_path\to\folder\the_local_1_the_data_file.xlsx",
    r"\\local_2\the_folder_path\to\folder\the_local_2_the_data_file.xlsx",
    r"\\local_3\the_folder_path\to\folder\the_local_3_the_data_file.xlsx",
    r"\\local_4\the_folder_path\to\folder\the_local_4_the_data_file.xlsx",
    r"\\local_5\the_folder_path\to\folder\the_local_5_the_data_file.xlsx",
    r"\\local_6\the_folder_path\to\folder\the_local_6_the_data_file.xlsx",
    r"\\local_7\the_folder_path\to\folder\the_local_7_the_data_file.xlsx",
    r"\\local_8\the_folder_path\to\folder\the_local_8_the_data_file.xlsx",
    r"\\local_9\the_folder_path\to\folder\the_local_9_the_data_file.xlsx"
    ]

for i, (odf_df, odf_path) in enumerate(zip(odf_lists, odf_filepaths), start=1):
    try:
        odf_df = pd.read_excel(odf_path)
        concat_odf.append(odf_df)
    except Exception as e:
        input(f"Failed to load: {e}\nPress Enter: Script will Terminate...")
        sys.exit(1)

merged_odf = pd.concat(concat_odf, ignore_index=True)

## Update the report
<div style="font-size: 14px;">
This is the key section of the project. First, it will ensure no whitespace in <i>keep_5.</i> Next, the reports are combined.<br>
Data is either discarded, copied, or added. If a value is on the new report, it is either added or triggers an update and/or copy<br>
from old report. If the value only exist on the old report, it will be discarded. We now have a new and updated dataframe to work with.</div>

In [ ]:
df['keep_5 #'] = df['keep_5 #'].astype(str).str.strip()
merged_odf['keep_5 #'] = merged_odf['keep_5 #'].astype(str).str.strip()

copy_columns = ['new_col_1', 'new_col_2']

for c in copy_columns:
    compared_df = df.merge(merged_odf[['keep_5 #', c]], on='keep_5 #', how='left', suffixes=('', '_pd'))
    compared_df[c] = compared_df[f'{c}_pd'].combine_first(compared_df[c])
    compared_df.drop(columns=[f'{c}_pd'], inplace=True)
    df = compared_df

new_df = compared_df

## Filter and send out the updated reports
<div style="font-size: 14px;">The new df will now be filtered out into the individual dfs.<br>
The <i>for loop</i> will convert the newly updated reports to Excel files and send them to their home.</div>

In [ ]:
local_1 = new_df[new_df['keep_1'].str.contains('local_1', case=False)]
local_2 = new_df[new_df['keep_1'].str.contains('local_2', case=False)]
local_3 = new_df[new_df['keep_1'].str.contains('local_3', case=False)]
local_4 = new_df[new_df['keep_1'].str.contains('local_4', case=False)]
local_5 = new_df[new_df['keep_1'].str.contains('local_5', case=False)]
local_6 = new_df[new_df['keep_1'].str.contains('local_6', case=False)]
local_7 = new_df[new_df['keep_1'].str.contains('local_7', case=False)]
local_8 = new_df[new_df['keep_1'].str.contains('local_8', case=False)]
local_9 = new_df[new_df['keep_1'].str.contains('local_9', case=False)]

location_list = (local_5, local_1, local_4, local_3, local_7, local_6, local_2, local_8, local_9)


for i, (write_df, write_path) in enumerate(zip(location_list, odf_filepaths), start=1):
    try:
        write_df.to_excel(write_path, index=False)
    except Exception as e:
        input(f"Failed to write: {e}\nPress Enter: Script may Terminate\nor fail to write noted error file(s)...")

## Format the Excel workbooks
<div style="font-size: 14px;">
Finally, the code will go out to each report and format the Excel sheet. This will ensure that the files are clean<br>
and readable when it is first opened. If the code ran completely, the print statement lets the user know the process completed<br>
and is now ready.</div>

In [ ]:
for i in odf_filepaths:
    
    wb = load_workbook(i)
    
    ws = wb["Sheet1"]
    
    ws.column_dimensions['A'].width = 20
    ws.column_dimensions['B'].width = 15
    ws.column_dimensions['C'].width = 15
    ws.column_dimensions['D'].width = 10
    ws.column_dimensions['E'].width = 10
    ws.column_dimensions['F'].width = 12
    ws.column_dimensions['G'].width = 10
    ws.column_dimensions['H'].width = 15
    ws.column_dimensions['I'].width = 10
    ws.column_dimensions['J'].width = 45
    ws.column_dimensions['K'].width = 20


    align_style = Alignment(horizontal="left", vertical="center", wrap_text=True)
    
    for row in ws.iter_rows(min_col=1, max_col=10):
        for cell in row:
            cell.alignment = align_style
    
    
    wb.save(i)

print('\n\n>>> Your files should be ready!')
input('>>>  Press ENTER to exit')

## In closing:
<div style="font-size: 14px;">
The output is a clean and simple report with information that is relevant only to the location the file was associated with.<br>
It can be ran daily and within minutes. Doing this manually would be resource intensive and would make daily updates impossible.<br>
<br>
I designed this project as a solution to a real world problem. Below are examples of what the pre and post code workbooks<br>
would have looked like. Remember the output would have been x9 with info relevant to each of the locations.</div>

## Pre run:
![](images/pre_run.png)

## Post run:
![](images/post_run.png)